In [1]:
import  torch
import  torch.nn as nn
import  torch.nn.functional as F
import  torch.optim as optim
from    torchvision import datasets, transforms
from visdom import Visdom
import numpy as np

In [2]:
 '''
 构建简单的模型:简单线性层+Relu函数的多层感知机
 '''
class MLP(nn.Module):

    def __init__(self):
        super(MLP, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(784, 200),
            nn.ReLU(inplace=True),
            nn.Linear(200, 200),
            nn.ReLU(inplace=True),
            nn.Linear(200, 10),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        x = self.model(x)
        return x

In [3]:
batch_size = 128
learning_rate = 0.01
epochs = 10

In [4]:
train_loader = torch.utils.data.DataLoader(datasets.MNIST(
    'data', 
    train=True,
    download=True,
   transform=transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.1307, ), (0.3081, ))])),
                                           batch_size=batch_size,
                                           shuffle=True)

In [5]:
test_loader = torch.utils.data.DataLoader(datasets.MNIST(
    'data',
    train=False,
    transform=transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.1307, ), (0.3081, ))])),
                                          batch_size=batch_size,
                                          shuffle=True)

In [6]:
# 注意此处初始化visdom类
viz = Visdom()
# 绘制起点
viz.line([0.], [0.], win="train loss", opts=dict(title='train_loss'))
#device = torch.device('cuda:0')
device = torch.device('cpu')
net = MLP().to(device)
#优化器和损失函数
optimizer = optim.SGD(net.parameters(), lr=learning_rate)
criteon = nn.CrossEntropyLoss().to(device)

Setting up a new session...


In [7]:
for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.view(-1, 28 * 28)
        #data, target = data.to(device), target.cuda()
        data, target = data.to(device), target
        logits = net(data)
        loss = criteon(logits, target)

        optimizer.zero_grad()
        loss.backward()
        # print(w1.grad.norm(), w2.grad.norm())
        optimizer.step()

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.view(-1, 28 * 28)
        #data, target = data.to(device), target.cuda()
        data, target = data.to(device), target
        logits = net(data)
        test_loss += criteon(logits, target).item()

        pred = logits.argmax(dim=1)
        correct += pred.eq(target).float().sum().item()

    test_loss /= len(test_loader.dataset)
    # 绘制epoch以及对应的测试集损失loss
    viz.line([test_loss], [epoch], win="train loss", update='append')
    print(
        '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))
            

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.312945
Train Epoch: 0 [12800/60000 (21%)]	Loss: 2.019008
Train Epoch: 0 [25600/60000 (43%)]	Loss: 1.306541
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.838697
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.534596

Test set: Average loss: 0.0037, Accuracy: 8777.0/10000 (88%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.573818
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.391589
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.435739
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.279797
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.283545


KeyboardInterrupt: 